In [28]:
import pandas as pd
import openpyxl
import json
import os
import re
from pandas import ExcelWriter
import xlwt


In [29]:
def findKeys(lang):
    with open('./../../../crowdsource-ui/locales/{lang}.json'.format(lang=lang)) as f:
        js = json.load(f)
    
    values = []
    for key, value in js.items():
        values.append(value)
    return values

In [30]:
def extract_tags(text):
    allowed_replacements = ["x", "y", "z", "u","v","w", "a-tag-replacement"]
    regex = r"<(\S*?)[^>]*>.*?<\/\1>|<.*?\/>"
    out_txt = text
    matches = re.finditer(regex, out_txt, re.MULTILINE)
    index = 0
    matched_tags_dict = {}
    for match in matches:
        matched_tag = match.group()
        if "<b>" in matched_tag:
            continue
        if "<a" in matched_tag:
            attrs = matched_tag[matched_tag.find('<a')+2: matched_tag.find('>')]
            matched_tag_replacement = matched_tag.replace(attrs,"")
            out_txt = out_txt.replace(matched_tag, matched_tag_replacement)
            matched_tags_dict['a-tag-replacement'] = attrs
        else:
            replacement = allowed_replacements[index]
            out_txt = out_txt.replace(matched_tag, '<{}>'.format(replacement))
            index+=1
            matched_tags_dict[replacement] = matched_tag
    return out_txt , matched_tags_dict

In [31]:
def save_xls(list_dfs, xls_path):
    with ExcelWriter(xls_path) as writer:
        for lang_code, lang_df in list_dfs.items():
            lang_df.to_excel(writer,lang_code)
        writer.save()
    print('done')

In [33]:
def get_dict_for_data(key, text, language, translation, extracted_tags):
    out_dict = {}
    out_dict["Actual key"] = [key]
    out_dict["English copy"] = [text]
    out_dict[language] = [translation]
    
    return out_dict

In [36]:
def create_all_trans():
    languages = [('hi', "Hindi"),('gu', "Gujarati"),('as', "Assamese"),('bn','Bengali'),('ta',"Tamil"),
                 ('te',"Telugu"),('mr',"Marathi"),('pa',"Punjabi"),('ml',"Malayalam"),('or',"Odia"),('kn',"Kannada")]
    eng_locale = 'en'
    english = 'English'
    all_dfs = {}
    for lang_code, lang_name in languages:
        lang_df = pd.DataFrame([], columns=[])
        with open('./../../../crowdsource-ui/locales/en.json') as f:
            eng_data = json.load(f)

        with open('./../../../crowdsource-ui/locales/{lang}.json'.format(lang=lang_code)) as f:
            lang_data = json.load(f)

        for key in eng_data.keys():
            processed_eng_text, extracted_tags = extract_tags(eng_data[key])

            processed_lang_text, tags = extract_tags(lang_data[key])

            data_dict = get_dict_for_data(key, processed_eng_text,lang_name, processed_lang_text, extracted_tags)

            try:
                tmp_df = pd.DataFrame.from_dict(data_dict, orient='columns')
                lang_df = lang_df.append(tmp_df, ignore_index=True)
            except Exception as e:
                print(e, "\n", data_dict, "\n\n")
    #     lang_df[lang_name] = ""
        all_dfs[lang_code] = lang_df
    return all_dfs

In [37]:
create_all_trans()
save_xls(all_dfs, 'all-trans.xls')

done


/Users/sreejithv/miniconda3/envs/jupyter/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  exec(code_obj, self.user_global_ns, self.user_ns)
